# Cap 2 - Registration

Point cloud registration is the process of aligning two or more 3D point clouds collected from different locations of the same scene. Here we present Iterative Closest Point Algorithm (ICP) registration. A method used to align two point clouds iteratively by minimizing the distance between their corresponding points. It is commonly used for 3D Point Cloud Registration tasks, such as aligning point clouds from LiDAR sensors or depth cameras. ICP can work well in environments with limited feature points, such as indoor environments or dense forests and is known as standard algorithm for obtain rigid body transformation in practice.

## Vanilla ICP based on SVD

### 1) ICP precursor - point clouds with corresponding points



#### a) Formal Problem Definition

- Let, $a_i$ be a point $i$ of set $a$ (with $n_a$ points), and $b_i$ a point $i$ on set $b$ (with $n_b$ points), homologous with $a_i$;

- We wish to find $R$ and $t$ of the rigid body transform: $a^{new}_i = Ra_i +t$;

- That minimizes the error function:

$$
e = \sum|| Ra_i +t - b_i||^2 = min  \ \ \ \ \ \ \ \ (1)
$$



#### b) Direct Optimal Solution

A solution to equation (1) with no initial guess and better than anyother is:

- Shift all poits to be centered in its center of mass;

- Perform a rotational alingment using Singular Value Decomposition (SVD).

In math terms, first we do a centroid calculation:

$$
\overline{a} = \frac{1}{n_a}\sum a_i \\
\overline{b} = \frac{1}{n_b}\sum b_i
$$

Do the shift:

$$
a'_i = a_i - \overline{a}  \\
b'_i = b_i - \overline{b}
$$

Compute Cross covariance matrix $H$:

$$
H = \sum a'_i{b'}^T_i
$$

Do a SVD on $H$:

$$
U\Sigma V^T = H
$$

The optimal $R$ ([proof here](https://youtu.be/dhzLQfDBx2Q?si=FDIqaFbgLxZkiamq&t=2516)) is found to be:

$$
R = VU^T
$$

Then, optimal $t$ is:

$$
t = \overline{b} - R\overline{a}
$$


### 2) ICP without corresponding points





#### a) Formal Problem Definition

- Let, $a$ be a point $i$ of set $a$ (with $n_a$ points), and $b_k$ a point $k$ on set $b$ (with $n_b$ points);

- We have a initial guess or some N point correspondeces;

- We wish to iteratively find the association $a_i$-$b_i$ and the transformation $R$ and $t$ of the rigid body transform: $a^{new}_i = Ra_i +t$;

- That minimizes the error function:

$$
e = \sum|| Ra_i +t - b_i||^2 = min  \ \ \ \ \ \ \ \ (1)
$$


#### b) Iterative Solution

- Pick for every point its closest neighnoor in the other point cloud;

- Compute the rigid body trasnform and align using SVD;

- Repeat till convergence.

In math terms, first we do a centroid calculation:

$$
\overline{a} = \frac{1}{n_a}\sum a_i \\
\overline{b} = \frac{1}{n_b}\sum b_i
$$

Do the shift:

$$
a'_i = a_i - \overline{a} \\
b'_i = b_i - \overline{b}
$$

Compute correspondences with nearest neighboor:

```python
correspondences = []
for k in range(n_b):
  distmin = 1000000000
  for i in range(n_a):
    dist = np.linalg.norm(a[i]-b[k])
    if dist<distmin:
      distmin = dist
      idx = i
  correspondences.append( (k,idx,distmin) )
```

Compute Cross covariance matrix $H$:

$$
H = \sum a'_i {b'}^T_i
$$

Do a SVD on $H$:

$$
U\Sigma V^T = H
$$

The optimal $R$ is found to be:

$$
R^{new} = VU^T
$$

Then, optimal $t$ is:

$$
t^{new} = \overline{b} - R^{new}\overline{a}
$$

If it's not the first iteration, remember to stack the transformations:

$$
R = R \cdot R^{new} \\
t = t+t^{new} \\
$$

Update $a$ in this iteration:

$$
a^{new}_i =  R{a_i} + t
$$

Recenter this new $a$:

$$
a'_i = a^{new}_i - \overline{a} \\
$$

Compute error:

$$
error = || a' - b' ||
$$

If error do not reach the tolerance, repeat 1st step.


### 3) Problems with the Vanilla ICP


- May require many iterations;
- Bad correspondentes degrade the results or stop them at local minima.



### 4) ICP variants


- Non linear Least squares based ICP
- Using point to plane metric with Least Squares ICP
- Different data associations strategies
- Weight the correspondences, i.e:

Distant points will have less importance and the mean becames weighted mean.

| b   | closest a | distance |  weight function (w) | normalized weight(p) |
| --- | --------- |--------- | -------     | -------------------- |
| 1   | 2         | 320      | 1/320       |  (1/320)/$\sum w_i$  |
| 2   | 2         | 227      | 1/227       |  (1/227)/$\sum w_i$  |
| 3   | 2         | 120      | 1/120       |  (1/120)/$\sum w_i$  |
| 4   | 7         | 310      | 1/310       |  (1/310)/$\sum w_i$  |
| 5   | 9         | 287      | 1/287       |  (1/287)/$\sum w_i$  |
| ... | ...       | ...      | ...         |  ...|
| n_b | i         | distmin  | 1/distmin   |  (1/distmin)/$\sum w_i$  |

$$
\overline{a} = \frac{\sum a_ip_i}{\sum p_i} \\
\overline{b} = \frac{\sum b_ip_i}{\sum p_i} \\
H = \sum a'_i {b'}^T_i p_i
$$

- remove outliers, i.e:

 Find and remove higher distances => Small but good number of matched data  => less computation with almost the same results.

| b   | closest a | distance |  remove  ? |
| --- | --------- |--------- | --------- |
| 1   | 2         | 320      | yes |
| 2   | 2         | 227      | yes |
| 3   | 2         | 120      | no |
| 4   | 7         | 310      | |
| 5   | 9         | 287      | |
| ... | ...       | ...      | |
| n_b | i         | distmin  | |

Also we could remove $t%$ of the higher distances. $t$ can be adaptative. Sometimes 10%, 20% or 50% dependig on situation. For example, if we have moving objetct between LiDAR point clouds, we will want for find and remove  these object points in correspondece or give to it less weight.
